# Topic Modeling on 'Russian Troll Tweets'

Does topic modeling using LDA make sense on short-form data such as tweets?
[Maybe yes](https://www.researchgate.net/post/What_is_a_good_way_to_perform_topic_modeling_on_short_text). Let's try.

In [1]:
import os
import json
import string
import warnings
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis
from pyLDAvis import gensim

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# displaying the vis right in our notebook
pyLDAvis.enable_notebook()

## Import what we'll work with

In [3]:
data_dir = 'data'
files = os.listdir(data_dir)

In [4]:
# change to loop over all of them
df = pd.read_csv(f"{data_dir}/{files[0]}")

## Inspect the data quickly

In [5]:
df.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,1.674084e+09,GAB1ALDANA,People are too toxic. I think I have people po...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2150,RETWEET,Hashtager,0,1,HashtagGamer
1,1.674084e+09,GAB1ALDANA,#NowPlaying Don't Shoot (I'm a Man) by @DEVO -...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2146,RETWEET,Hashtager,0,1,HashtagGamer
2,1.674084e+09,GAB1ALDANA,the 'I'm the most boring person in the world' ...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2159,RETWEET,Hashtager,0,1,HashtagGamer
3,1.674084e+09,GAB1ALDANA,#MyAchillesHeel slippery floors https://t.co/R...,United States,Norwegian,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2160,RETWEET,Hashtager,0,1,HashtagGamer
4,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Boring narcissists.....nothing...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2158,RETWEET,Hashtager,0,1,HashtagGamer


In [6]:
df.dtypes

external_author_id    float64
author                 object
content                object
region                 object
language               object
publish_date           object
harvested_date         object
following               int64
followers               int64
updates                 int64
post_type              object
account_type           object
new_june_2018           int64
retweet                 int64
account_category       object
dtype: object

In [7]:
df.describe()

,external_author_id,following,followers,updates,new_june_2018,retweet
count,3.884520e+05,388452.000000,388452.000000,388452.000000,388452.000000,388452.000000
mean,7.200485e+16,2486.855076,6820.914885,7590.116050,0.137391,0.602679
std,2.330445e+17,3804.948594,15269.406368,10951.128592,0.344261,0.489344
min,4.100056e+08,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.670448e+09,476.000000,474.000000,1550.000000,0.000000,0.000000
50%,2.439943e+09,1225.000000,1043.000000,3259.000000,0.000000,1.000000
75%,2.787256e+09,2545.000000,2272.000000,7493.000000,0.000000,1.000000
max,9.812510e+17,34338.000000,110155.000000,52860.000000,1.000000,1.000000


## Text preprocessing and data cleaning

I decided to avoid cleaning the text data of tweets, because punctuation and capitalization might play an important role. Maybe not a good idea, we'll see. Eventually it's possible to just go back to this point and do some cleaning before running the rest of the analysis.

Bad idea.

Here I am again. :)

In [8]:
def preprocess(text, add_punct=None, add_stopwords=None):
    # remove capitalization
    text = text.lower()
    # tokenize
    tokens = word_tokenize(text)
    # remove punctuation
    punct = string.punctuation
    # adding additional common punctuation chars of the texts, if provided
    if add_punct:
        punct += add_punct
    no_punct = [t for t in tokens if not t in punct]
    # remove stopwords
    stpw = stopwords.words('english')
    if add_stopwords:
        stpw.extend(add_stopwords)
    no_stpw = [t for t in no_punct if not t in stpw]
    # lemmatize remaining tokens
    lem = WordNetLemmatizer()
    lem_tokens = [lem.lemmatize(t) for t in no_stpw]
    return lem_tokens

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
tweet_corpus = [preprocess(tweet, add_punct="’“”–''''``—", add_stopwords=["http", "https"]) 
                for tweet in list(df.content) 
                if type(tweet) == str]

In [11]:
tweet_corpus[2:3]

[["'i",
  "'m",
  'boring',
  'person',
  'world',
  'starterpack',
  'pokemongo',
  '//t.co/u8woa1s3j7']]

## Topic modeling with `gensim`

In [12]:
dictionary = Dictionary(tweet_corpus)
gen_corpus = [dictionary.doc2bow(tweet) for tweet in tweet_corpus]

In [13]:
warnings.filterwarnings("ignore")
ldamodel = LdaModel(corpus=gen_corpus, num_topics=10, id2word=dictionary)

In [14]:
ldamodel.show_topics()

[(0,
  '0.080*"news" + 0.013*"police" + 0.012*"man" + 0.011*"u" + 0.009*"trump" + 0.008*"\'s" + 0.007*"obama" + 0.007*"state" + 0.007*"new" + 0.007*"politics"'),
 (1,
  '0.016*"pjnet" + 0.016*"trump" + 0.015*"hillary" + 0.014*"clinton" + 0.011*"\'s" + 0.010*"mt" + 0.009*"death" + 0.007*"maga" + 0.007*"medium" + 0.007*"amp"'),
 (2,
  '0.012*"vote" + 0.008*"health" + 0.007*"party" + 0.007*"�" + 0.006*"lie" + 0.006*"truth" + 0.006*"left" + 0.006*"question" + 0.006*"person" + 0.005*"conservative"'),
 (3,
  '0.025*"\'s" + 0.022*"trump" + 0.017*"n\'t" + 0.015*"..." + 0.009*"woman" + 0.009*"rt" + 0.008*"politics" + 0.007*"people" + 0.007*"say" + 0.006*"gun"'),
 (4,
  '0.012*"dead" + 0.011*"help" + 0.010*"show" + 0.010*"campaign" + 0.010*"4" + 0.010*"3" + 0.008*"election" + 0.008*"video" + 0.007*"ka" + 0.007*"boy"'),
 (5,
  '0.011*"law" + 0.010*"suspect" + 0.009*"arrested" + 0.007*"kc" + 0.006*"new" + 0.006*"charge" + 0.006*"sander" + 0.005*"post" + 0.005*"city" + 0.005*"issue"'),
 (6,
  '0.02

## Visualize the resulting topics

Here we are using `pyLDAvis` - the `import` needs to be explicit to work, so it might appear that this is part of the `gensim` package, but instead this is from `pyLDAvis`.

In [15]:
gensim.prepare(ldamodel, gen_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.294629  0.034950       1        1  17.254362
3     -0.257286  0.077717       2        1  16.933910
6      0.152432  0.250683       3        1  14.390466
1     -0.152392 -0.034920       4        1   9.541903
2      0.098880 -0.055354       5        1   8.686483
4      0.059844 -0.122204       6        1   7.368148
7      0.071942 -0.114706       7        1   7.136994
5      0.058839 -0.171556       8        1   6.844178
8      0.127661  0.049126       9        1   5.989678
9      0.134709  0.086264      10        1   5.853884, topic_info=      Category          Freq        Term         Total  loglift  logprob
term                                                                    
1660   Default  51155.000000        news  51155.000000  30.0000  30.0000
9048   Default  15673.000000       kansa  15673.000000  29.0000  29.0000
6762   Default  13825.000000           в  13825.000000  28.0000  28.0000
53     Default  24547.000000          's  24547.000000  27.0000  27.0000
97     Default  25591.000000       trump  25591.000000  26.0000  26.0000
3532   Default   7662.000000           ‘   7662.000000  25.0000  25.0000
10     Default  12046.000000         n't  12046.000000  24.0000  24.0000
6774   Default   7503.000000           и   7503.000000  23.0000  23.0000
24     Default  11328.000000         ...  11328.000000  22.0000  22.0000
16749  Default   6254.000000       pjnet   6254.000000  21.0000  21.0000
603    Default   9748.000000      police   9748.000000  20.0000  20.0000
31     Default   8838.000000     hillary   8838.000000  19.0000  19.0000
52     Default   7196.000000           u   7196.000000  18.0000  18.0000
6903   Default   6310.000000          на   6310.000000  17.0000  17.0000
3007   Default   4226.000000         isi   4226.000000  16.0000  16.0000
9      Default   9519.000000         man   9519.000000  15.0000  15.0000
1073   Default   8767.000000     clinton   8767.000000  14.0000  14.0000
74     Default   7808.000000       woman   7808.000000  13.0000  13.0000
601    Default   5209.000000        city   5209.000000  12.0000  12.0000
305    Default   9404.000000    politics   9404.000000  11.0000  11.0000
6788   Default   4944.000000          не   4944.000000  10.0000  10.0000
176    Default   5372.000000          rt   5372.000000   9.0000   9.0000
304    Default   4587.000000        vote   4587.000000   8.0000   8.0000
101    Default   3162.000000         car   3162.000000   7.0000   7.0000
776    Default   3204.000000        dead   3204.000000   6.0000   6.0000
1121   Default   7663.000000         say   7663.000000   5.0000   5.0000
1585   Default   3110.000000        help   3110.000000   4.0000   4.0000
6780   Default   4096.000000           с   4096.000000   3.0000   3.0000
3641   Default   3378.000000          mt   3378.000000   2.0000   2.0000
0      Default   6908.000000      people   6908.000000   1.0000   1.0000
...        ...           ...         ...           ...      ...      ...
108    Topic10   1562.376221    business   1563.201416   2.8375  -4.9333
3562   Topic10   1548.748413    congress   1549.573608   2.8375  -4.9420
8002   Topic10   1465.871094    targeted   1466.696899   2.8375  -4.9970
10437  Topic10   1285.121338    district   1285.946533   2.8374  -5.1286
3462   Topic10   1113.447144    military   1114.272339   2.8373  -5.2720
8330   Topic10    987.865601    brussels    988.690735   2.8372  -5.3917
1049   Topic10    902.472839        jail    903.297974   2.8372  -5.4821
836    Topic10    878.605835        past    879.430969   2.8371  -5.5089
6635   Topic10    827.967651    violence    828.792786   2.8371  -5.5683
1150   Topic10    823.733887        drop    824.559021   2.8371  -5.5734
1677   Topic10    775.131653       using    775.956787   2.8370  -5.6342
3582   Topic10    701.729492        test    702.554626   2.8369  -5.7337
11677  Topic10    697.01

## Ideas / Todos

* run this analysis on the full corpus (one file takes approximately 8-9 minutes to process)
* cherry-pick some words to include into the stopword list
* adapt the tokenizer to avoid splitting `#` symbols from twitter hashtags